In [ ]:
#default_exp modelling.mybifacialvf
#default_cls_lvl 3

In [ ]:
#export
from capetools.imports import *
import bifacialvf
import pvlib

# bifacialvf
> A wrapper around NREL's bifacialvf

Let's put some gps data

In [ ]:
#export
CHAMBERY = {'Name':'Chambery', 'latitude': 45.637001, 'longitude': 5.881, 'Elevation': 235.0, 'TZ':-1.0}

bifacialvf has hard-coded column names 🤣

In [ ]:
DATAPATH = Path.cwd().parent/'data'
DATAPATH.ls()

(#1) [Path('/home/tc256760/Documents/capetools/data/sample_data.hdf')]

In [ ]:
df = pd.read_hdf(DATAPATH/'sample_data.hdf')
df.head()

,temp_air,wind_speed,wind_dir,ghi,dni,dhi,ghi30,ref_cell_30,pressure,humidity
Date UTC,,,,,,,,,,
2019-01-01 00:00:00,4.39,1.87,230.1,-0.83,0.0,-0.83,-1.17,0.0,1004.92,86.8
2019-01-01 01:00:00,4.33,2.87,228.9,-0.93,0.0,-0.83,-1.06,0.0,1004.94,86.8
2019-01-01 02:00:00,4.29,2.86,229.4,-0.83,0.0,-0.83,-1.17,0.0,1004.68,86.7
2019-01-01 03:00:00,4.15,2.04,186.0,-0.83,0.0,-0.83,-1.06,0.0,1003.99,85.7
2019-01-01 04:00:00,3.85,1.91,200.9,-0.72,0.0,-0.83,-0.85,0.0,1003.76,87.5


In [ ]:
CHAMBERY['latitude']

45.637001

In [ ]:
sunpos = pvlib.solarposition.get_solarposition(df.index, 
                                               CHAMBERY['latitude'], 
                                               CHAMBERY['longitude'], 
                                               CHAMBERY['Elevation'],
                                               df.pressure)
sunpos.head()

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
Date UTC,,,,,,
2019-01-01 00:00:00,157.028814,157.028814,-67.028814,-67.028814,12.053823,-3.195874
2019-01-01 01:00:00,152.142520,152.142520,-62.142520,-62.142520,42.533373,-3.215687
2019-01-01 02:00:00,143.742340,143.742340,-53.742340,-53.742340,63.390006,-3.235491
2019-01-01 03:00:00,133.848157,133.848157,-43.848157,-43.848157,78.092116,-3.255288
2019-01-01 04:00:00,123.435454,123.435454,-33.435454,-33.435454,89.729489,-3.275076


In [ ]:
pd.concat([df, sunpos], axis=1)

,temp_air,wind_speed,wind_dir,ghi,dni,dhi,ghi30,ref_cell_30,pressure,humidity,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
Date UTC,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,4.39,1.87,230.10,-0.83,0.00,-0.83,-1.17,0.0,1004.92,86.8,157.028814,157.028814,-67.028814,-67.028814,12.053823,-3.195874
2019-01-01 01:00:00,4.33,2.87,228.90,-0.93,0.00,-0.83,-1.06,0.0,1004.94,86.8,152.142520,152.142520,-62.142520,-62.142520,42.533373,-3.215687
2019-01-01 02:00:00,4.29,2.86,229.40,-0.83,0.00,-0.83,-1.17,0.0,1004.68,86.7,143.742340,143.742340,-53.742340,-53.742340,63.390006,-3.235491
2019-01-01 03:00:00,4.15,2.04,186.00,-0.83,0.00,-0.83,-1.06,0.0,1003.99,85.7,133.848157,133.848157,-43.848157,-43.848157,78.092116,-3.255288
2019-01-01 04:00:00,3.85,1.91,200.90,-0.72,0.00,-0.83,-0.85,0.0,1003.76,87.5,123.435454,123.435454,-33.435454,-33.435454,89.729489,-3.275076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,-0.84,0.78,23.88,-1.36,0.09,-1.36,-2.09,-0.1,1005.05,100.0,120.109907,120.109907,-30.109907,-30.109907,266.843772,-2.982273
2019-12-31 20:00:00,-0.70,0.16,193.80,-1.36,-0.09,-1.48,-1.76,-0.1,1005.08,100.0,130.567903,130.567903,-40.567903,-40.567903,277.905352,-3.002181
2019-12-31 21:00:00,-1.24,0.42,156.60,-1.48,-0.09,-1.63,-1.76,0.0,1004.99,100.0,140.699448,140.699448,-50.699448,-50.699448,291.371504,-3.022080


In [ ]:
#export
def get_tmy3(data: DataFrame):
    "Rename to upper case dni and ghi cols"
    tmy3 = (data[['dni', 'dhi', 'zenith', 'azimuth', 'elevation']]
            .rename(columns={'dni':'DNI', 'dhi':'DHI'}))
    return tmy3

In [ ]:
data = get_tmy3(df)

KeyError: "['zenith', 'elevation', 'azimuth'] not in index"

We will need to compare to pvfactors output, so we create a function to do that:

In [ ]:
#export
def format_output_as_pvfactor(res: DataFrame, cuts: int): 
    "Formats output from bifacialvf as pvfactor's one"
    front_cols = [f'No_{i+1}_RowFrontGTI' for i in range(cuts)]
    back_cols = [f'No_{i+1}_RowBackGTI' for i in range(cuts)]
    aux = pd.DataFrame(index=res.index)
    aux['qinc_front'] = res[front_cols].mean(axis=1)
    aux['qinc_back_mean'] = res[back_cols].mean(axis=1)
    aux[back_cols] = res[back_cols]
    aux =  aux.rename(columns=dict(zip(back_cols,[f'qinc_back_{i}' for i in range(cuts-1, -1, -1)] )))
    return aux[['qinc_front', 'qinc_back_mean']+[f'qinc_back_{i}' for i in range(cuts)]]

## Engine

In [ ]:
#export
def bifacialvf_engine_run(data,  pvarray_parameters, gps_data=CHAMBERY):
    "Run bifacialvf on data, with pvarray_parameters at location"
    tmy3 = get_tmy3(data)
    outfile = 'output.csv'
    bifacialvf.simulate(tmy3, gps_data, outfile, **pvarray_parameters)
    return (pd.read_csv(outfile, header=2, index_col='date', parse_dates=True)
            .pipe(format_output, cuts=pvarray_parameters['cellRows']))

In [ ]:
bifacialvf_engine_run()

# Exports -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()